In [1]:
%matplotlib inline
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle
import glob

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np, to_var
from object_pose_utils.utils.display import *

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import warnings
warnings.filterwarnings('ignore')

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


## Dataset Information

In [2]:
dataset_root = '/ssd0/datasets/ycb/YCB_Video_Dataset'

non_sym_objs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 17, 18,]
sym_objs = [13, 16, 19, 20, 21]
textureless_objects = set([10, 11, 13, 14, 16, 19, 20, 21])
textured_objects = set(range(1,22)) - set([10, 11, 13, 14, 16, 19, 20, 21])


with open('{0}/image_sets/classes.txt'.format(dataset_root)) as f:                                    
    classes = f.read().split()
classes.insert(0, '__background__')

In [3]:
def recursiveUpdate(dict_base, dict_new):
    for k,v in dict_new.items():
        if(k in dict_base):
            dict_base[k].update(v)
        else:
            dict_base[k] = v

def updateResults(results_dict, new_results_filename):
    new_data = np.load(new_results_filename, allow_pickle=True)
    new_results = {k:v.item() for k,v in new_data.items()}
    recursiveUpdate(results_dict, new_results)

In [4]:
from object_pose_utils.utils.pose_error import accuracyAUC

def non_func(x):
    return x

def mean_abs(x):
    return np.mean(np.abs(x))

def log_clean(x):
    return np.log(np.maximum(1e-6, x))

def mean_log_clean(x):
    return np.mean(log_clean(x))

def mean_log(x):
    return np.mean(np.log(x))   

def accuracyAUC100(x, max_theshold):
    return accuracyAUC(x, max_theshold)*100

def makeTableEntries(data_dict, keys = None, objs = list(range(1,22)), 
              val_func = non_func, bold_func = max,
              individual = False,
              agg_title = 'All'):
    if(keys is None):
        keys = list(data_dict.keys())
    
    data_str = ''
    if(individual):
        for obj in objs:    
            data_str += ' '.join(classes[obj].split('_')[1:]) + '\n'
            vals = [val_func(np.array(data_dict[k][obj])) for k in keys]
            bold_val = bold_func(vals)
            for v in vals:
                if(v == bold_val):
                    data_str += ' & \\textbf{' + '{:0.2f}'.format(v) + '}'
                else:
                    data_str += ' & {:0.2f}'.format(v)
            data_str += ' \\\\'
            data_str += '\n'

        data_str += '\hline\n'

    data_str += '{} \n'.format(agg_title)
    vals = [val_func(np.concatenate([np.array(data_dict[k][o]) for o in objs])) for k in keys]
    bold_val = bold_func(vals)
    for v in vals:
        if(v == bold_val):
            data_str += ' & \\textbf{' + '{:0.2f}'.format(v) + '}'
        else:
            data_str += ' & {:0.2f}'.format(v)
    data_str += ' \\\\'
    data_str += '\n'
    #print(data_str)
    return data_str
    
def makeTableHeading(names, heading = None):
    data_str = ''
    data_str +='\\begin{table*}[h!]\n'
    data_str +='\\small\n'
    data_str +='\\centering\n'
    data_str +='\\begin{tabular} {||c||' + len(names)*'c|' +'|}\n'
    data_str +='\\hline\n'
        
    data_str += ' '
    if(heading is not None):
        data_str += heading
    else:
        for k in names:
            data_str += ' & ' + k
        data_str += '\\\\'
    data_str += '\n'

    data_str += '\hline\n'
    
    #print(data_str)
    return data_str

def makeTableEnding(caption = None, label = None):
    data_str = ''
    data_str +='\\hline\n'
    data_str +='\\end{tabular}\n'
    if(caption is not None):
        data_str +='\\caption{' + caption + '}\n'
    if(label is not None):
        data_str +='\\label{tbl:' + label + '}\n'
    data_str +='\\end{table*}\n'
    #print(data_str)
    return data_str

In [5]:
from generic_pose.losses.bingham_loss import isoLikelihood, duelLikelihood

In [6]:
class mLikelihood(object):
    def __init__(self, M, Z):
        self.M = M
        self.Z = Z
    def __call__(self, quats):
        return 2*bingham_likelihood(self.M, self.Z, quats.cuda())

In [33]:
results_1 = 'ral_results/pcnn_single_img_hist_cosine_results.npz'
results_2 = 'ral_results/pcnn_single_img_fused_hist_reg_fc6_hist_cosine_hist_uniform_bing_fixed_results.npz'
results_3 = 'ral_results/pcnn_single_img_hist_comp_fc6_results.npz'
results_4 = 'ral_results/pcnn_single_img_hist_conf_bing_iso_bing_duel_results.npz'
results_5 = 'ral_results/pcnn_single_img_bing_dropout_results.npz'

results_bing_reg = 'ral_results/final_pcnn_single_img_bing_iso_bing_duel_results.npz'
results_fixed_bing = 'ral_results/final_pcnn_single_img_bing_fixed_results.npz'

results = {}
updateResults(results, results_fixed_bing)
updateResults(results, results_bing_reg)
updateResults(results, results_2)
updateResults(results, results_1)
updateResults(results, results_3)
updateResults(results, results_4)
updateResults(results, results_5)

for res_k, res_v in results.items():
    for alg_k, alg_v in res_v.items():
        for k, v in alg_v.items():
            if(type(v) is dict):
                results[res_k][alg_k][k] = list(v.values())

In [34]:
results.keys()

dict_keys(['likelihood', 'sym_angular_error', 'add_error', 'add_sym_error', 'sym_angular_error_mode', 'add_error_mode', 'add_sym_error_mode', 'lik_distribution'])

In [35]:
results_keys = ['likelihood', 'sym_angular_error_mode', 'add_error_mode', 'add_sym_error_mode']
algorithm_keys = ['hist_uniform',
                  'hist_conf', 
                  'hist_reg_fc6', 
                  'hist_comp_fc6', 
                  'hist_cosine', 
                  'bing_fixed',
                  'bing_dropout',
                  'bing_iso',
                  'bing_duel',]

In [36]:
for res_k in results.keys():
    for alg_k in algorithm_keys:
        if(alg_k not in results[res_k].keys()):
            results[res_k][alg_k] = {}
            for k in range(1,22):
                results[res_k][alg_k][k] = [np.nan]

In [37]:
table_heading = """ &  & \multicolumn{4}{c|}{Histogram} & \multicolumn{4}{c||}{Bingham} \\\\\n\\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Dropout & Reg Iso & Reg Full \\\\"""

""" 
 &  & \\multicolumn{4}{c||}{Histogram} & \\multicolumn{4}{c||}{Bingham} \\\
\\hline
 &  & Stat & \\multicolumn{3}{c||}{Learned} & \\multicolumn{2}{c||} Stat & \\multicolumn{2}{c||} Learned \\\
\\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Dropout & Reg Iso & Reg Full \\\
""";

In [38]:
def median_log(x):
    return np.median(np.log(x))

In [39]:
median_log(results['likelihood']['bing_iso'][1])

1.7405415391530707

In [40]:
all_item_tables = {}
from functools import partial

#for k, v in results.items():
for k in results_keys:
    v = results[k]
    table_str = ''
    table_str += makeTableHeading([h.replace('_', ' ') for h in v.keys()],
                                  heading = table_heading)
    if(k == 'likelihood'):
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = non_sym_objs, 
                                      val_func=mean_log_clean, bold_func = max,
                                      agg_title = 'Non-Symmetric-PC')
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = sym_objs,
                                      val_func=mean_log_clean, bold_func = max, 
                                      agg_title = 'Symmetric-PC')
        table_str += "\\hline\n"
        table_str += makeTableEntries(v, keys=algorithm_keys, val_func=mean_log_clean, bold_func = max)
        #table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
        table_str += makeTableEnding(caption = 'Mean Log Likelihood of Ground Truth Orientation')


    elif('angular' in k):
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = non_sym_objs, 
                                      val_func=partial(accuracyAUC100, max_theshold=180), bold_func = max,
                                      agg_title = 'Non-Symmetric-PC')
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = sym_objs, 
                                      val_func=partial(accuracyAUC100, max_theshold=180), bold_func = max, 
                                      agg_title = 'Symmetric-PC')
        #table_str += makeTableEntries(v, val_func=mean_abs, bold_func = min)
        table_str += "\\hline\n"
        table_str += makeTableEntries(v, keys=algorithm_keys, val_func=partial(accuracyAUC100, max_theshold=180), bold_func = max)
        #table_str += makeTableEnding(caption = 'AUC ' + k.replace('_', ' '))
        table_str += makeTableEnding(caption = 'AUC of Symmetric Angular Error of Distribution Mode')
    else:
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = non_sym_objs,
                                      val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max,
                                      agg_title = 'Non-Symmetric-PC')
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = sym_objs,
                                      val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max,
                                      agg_title = 'Symmetric-PC')
        table_str += "\\hline\n"
        #table_str += makeTableEntries(v, val_func=mean_abs, bold_func = min)
        table_str += makeTableEntries(v, keys=algorithm_keys, val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max)
        #table_str += makeTableEnding(caption = 'AUC ' + k.replace('_', ' '))
        if('sym' in k):
            table_str += makeTableEnding(caption = 'AUC of Symmetric Average Distance Error of Distribution Mode')
        else:
            table_str += makeTableEnding(caption = 'AUC of Average Distance Error of Distribution Mode')
    
    #table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
    all_item_tables[k] = table_str
    print(table_str)
    print('\n\n\n')

\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c|c|c|c|c||}
\hline
  &  & \multicolumn{4}{c|}{Histogram} & \multicolumn{4}{c||}{Bingham} \\
\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Dropout & Reg Iso & Reg Full \\
\hline
Non-Symmetric-PC 
 & -2.29 & -2.46 & 1.87 & 2.17 & -0.92 & -2.29 & \textbf{2.71} & 1.92 & 2.04 \\
Symmetric-PC 
 & -2.29 & -7.75 & -1.23 & \textbf{-0.52} & -1.55 & -2.29 & -5.28 & -8.79 & -3.73 \\
\hline
All 
 & -2.29 & -3.31 & 1.37 & \textbf{1.74} & -1.02 & -2.29 & 1.43 & 0.20 & 1.11 \\
\hline
\end{tabular}
\caption{Mean Log Likelihood of Ground Truth Orientation}
\end{table*}





\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c|c|c|c|c||}
\hline
  &  & \multicolumn{4}{c|}{Histogram} & \multicolumn{4}{c||}{Bingham} \\
\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Dropout & Reg Iso & Reg Full \\
\hline
Non-Symmetric-PC 
 & 23.06 & 68.16 & 79.93 & 80.62 & 84.37 & 85.88 & \textbf{86.38} & 85

In [14]:
all_item_tables = {}
from object_pose_utils.utils.pose_error import accuracyAUC
def accuracyAUC100(x, max_theshold):
    return accuracyAUC(x, max_theshold)*100

from functools import partial

#for k, v in results.items():
for k in results_keys:
    v = results[k]
    table_str = ''
    table_str += makeTableHeading([h.replace('_', ' ') for h in v.keys()], 
                                  heading = table_heading)
    if(k == 'likelihood'):
        table_str += makeTableEntries(v, keys=algorithm_keys, val_func=mean_log_clean, bold_func = max, individual = True)
        #table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
        table_str += makeTableEnding(caption = 'Mean Log Likelihood of Ground Truth Orientation')
    elif('angular' in k):
        #table_str += makeTableEntries(v, val_func=mean_abs, bold_func = min, individual = True)    
        table_str += makeTableEntries(v, keys=algorithm_keys, val_func=partial(accuracyAUC100, max_theshold=180), 
                                      bold_func = max, individual = True)
        #table_str += makeTableEnding(caption = 'AUC ' + k.replace('_', ' '))
        table_str += makeTableEnding(caption = 'AUC of Symmetric Angular Error of Distribution Mode')
    else:
        #table_str += makeTableEntries(v, val_func=mean_abs, bold_func = min, individual = True)
        table_str += makeTableEntries(v, keys=algorithm_keys, val_func=partial(accuracyAUC100, max_theshold=0.1), 
                                      bold_func = max, individual = True)
        #table_str += makeTableEnding(caption = 'AUC ' + k.replace('_', ' '))
        if('sym' in k):
            table_str += makeTableEnding(caption = 'AUC of Symmetric Average Distance Error of Distribution Mode')
        else:
            table_str += makeTableEnding(caption = 'AUC of Average Distance Error of Distribution Mode')
    
    all_item_tables[k] = table_str
    print(table_str)
    print('\n\n\n')


\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c|c|c|c|c||}
\hline
  &  & \multicolumn{4}{c|}{Histogram} & \multicolumn{4}{c||}{Bingham} \\
\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Dropout & Reg Iso & Reg Full \\
\hline
master chef can
 & -2.29 & -4.99 & 1.40 & \textbf{2.32} & -0.83 & -2.29 & 0.09 & -1.57 & -1.70 \\
cracker box
 & -2.29 & -6.06 & 0.92 & -0.09 & -1.06 & -2.29 & \textbf{3.71} & 0.90 & 1.81 \\
sugar box
 & -2.29 & 0.77 & 3.19 & 2.62 & -0.82 & -2.28 & 4.20 & \textbf{4.63} & 3.17 \\
tomato soup can
 & -2.29 & -0.21 & 2.14 & 2.52 & -0.82 & -2.29 & \textbf{3.99} & 1.78 & 3.45 \\
mustard bottle
 & -2.29 & -1.18 & 1.28 & 3.02 & -0.79 & -2.29 & \textbf{4.81} & 4.39 & 3.39 \\
tuna fish can
 & -2.29 & -4.24 & 0.91 & \textbf{2.64} & -1.12 & -2.29 & 1.23 & -0.12 & 0.74 \\
pudding box
 & -2.29 & -1.62 & 2.97 & 3.13 & -0.95 & -2.29 & \textbf{4.54} & 4.12 & 2.65 \\
gelatin box
 & -2.29 & -0.56 & 3.11 & 3.53 & -0.93 & -2.28 & 5.73 & \textbf{5.8

In [15]:
old_hist_results = 'ral_results/single_img_results.npz'
old_results = {}
updateResults(old_results, old_hist_results)

In [16]:
print(mean_log_clean(results['likelihood']['hist_conf'][19]))
print(mean_log_clean(results['likelihood']['hist_comp_global'][19]))


-9.343797087324491


KeyError: 'hist_comp_global'

In [ ]:
np.log(results['likelihood']['hist_comp_global'][19])